In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

root_dir = '/content/gdrive/MyDrive/Data'

Mounted at /content/gdrive


In [2]:
import os
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import RMSprop
from pandas.core.frame import DataFrame
from time import sleep

In [8]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     |████████████████████████████████| 85 kB 3.7 MB/s 
     |████████████████████████████████| 4.3 MB 7.5 MB/s 
     |████████████████████████████████| 98 kB 7.7 MB/s 


In [3]:
experience = 1

features_dir = os.path.join(root_dir, "18 participants/AX3/TransformedData" + str(experience))
file_name = os.path.join(features_dir, "Labelled")

def encode_sleep_stage(val):
  arr = {'N1':0, 'N2':1, 'N3':2, 'REM':3, 'Wake':4, 'Artefact':np.NaN, 'A':np.NaN}
  return arr[val]

def encode_sleep_state(val):
  arr = {'N1':0, 'N2':1, 'N3':2, 'REM':3, 'Wake':4, 'Artefact':np.NaN, 'A':np.NaN}
  return arr[val]

def plot_graph(x, y, z, x_label, y_label, legends, figsize=(7,7)):
  plt.plot(z, x, 'r--')
  plt.plot(z, y, 'b-')
  plt.legend(legends)
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  plt.figure(figsize=figsize)
  plt.show();

dataset = pd.read_pickle(file_name)
dataset['Sleep State'] = dataset['Sleep State'].apply(lambda x: encode_sleep_stage(x))
dataset['Sleep State'] = pd.Series(np.array(dataset['Sleep State'])).interpolate()
dataset['Sleep State'] = np.array(dataset['Sleep State'].replace(to_replace = np.nan, value=4), dtype=np.int64)

n = dataset.shape[0]
train_ratio = 0.9
separate_id = int(n * train_ratio)

labels = np.zeros((n,5))
y_single = np.zeros((n,2))

for i in range(n):
  label = dataset.iloc[i]['Sleep State']
  labels[i][label] = 1
  if label < 4:
    y_single[i][0] = 1
  else:
    y_single[i][1] = 1

print(dataset['Sleep State'].value_counts())
dataset = dataset.drop(['Label', 'Sleep State'], axis=1)
dataset += 0.001
dataset = np.array(dataset)

dataset = dataset.reshape((dataset.shape[0], dataset.shape[1], 1, 1))

x_train = dataset[:separate_id]
y_train = labels[:separate_id]
y_train_single = y_single[:separate_id]

x_test = dataset[separate_id:]
y_test = labels[separate_id:]
y_test_single = y_single[separate_id:]

print(dataset.shape)
#print(dataset[0])
print(labels.shape)

1    6536
4    5670
2    2740
0    2140
3    2123
Name: Sleep State, dtype: int64
(19209, 24, 1, 1)
(19209, 5)


In [1]:
from tensorflow.keras.layers import Input, Conv2D, Activation, MaxPool2D, Flatten, Add, Dense

input_ = Input(shape=(dataset.shape[1],1,1), name='input')

conv_1 = Conv2D(24, 1, name='conv_1')(input_)
act_1 = Activation('relu', name='act_1')(conv_1)

conv_2 = Conv2D(16, 1, name='conv_2')(act_1)
act_2 = Activation('relu', name='act_2')(conv_2)

flat_1 = Flatten(name='flat_1')(act_2)

task_1_dense_1 = Dense(10, activation='sigmoid', name='task_1_dense_1')(flat_1)
task_1_dense_2 = Dense(5, activation='softmax', name='task_1_dense_2')(task_1_dense_1)

task_2_dense_1 = Dense(10, activation='sigmoid', name='task_2_dense_1')(flat_1)
task_2_dense_2 = Dense(2, activation='softmax', name='task_2_dense_2')(task_2_dense_1)

model = tf.keras.models.Model(input_, [task_1_dense_2, task_2_dense_2])

model.compile(
    loss={
        'task_1_dense_2': 'categorical_crossentropy',
        'task_2_dense_2': 'binary_crossentropy'
    },
    optimizer='adam',
    metrics=['accuracy']
)

_ = model.fit(x_train, [y_train, y_train_single], validation_data=(x_test, [y_test, y_test_single]), 
              epochs=10)

NameError: ignored